In [9]:
from pathlib import Path
from pyspark.sql import functions as F, Window

# =========================
# Paths
# =========================

PROJECT_ROOT = Path().resolve().parents[0]
GOLD_ROOT = (PROJECT_ROOT / "lakehouse" / "gold").resolve()
GOLD_PATH = (GOLD_ROOT / "boi_ipca").resolve()

INSIGHTS_PATH = (GOLD_ROOT / "insights").resolve()
ANALITICO_PATH = (GOLD_ROOT / "analitico").resolve()

print("BOI_IPCA_GOLD :", GOLD_PATH)
print("INSIGHTS_PATH_GOLD :", INSIGHTS_PATH)
print("ANALITICO_PATH_GOLD :", ANALITICO_PATH)

BOI_IPCA_GOLD : C:\Users\fdani\project_ipca_boi\lakehouse\gold\boi_ipca
INSIGHTS_PATH_GOLD : C:\Users\fdani\project_ipca_boi\lakehouse\gold\insights
ANALITICO_PATH_GOLD : C:\Users\fdani\project_ipca_boi\lakehouse\gold\analitico


In [4]:
# Abertura de sessão Spark para Camada GOLD: Configuação

from pyspark.sql import SparkSession
from delta import configure_spark_with_delta_pip
import os, sys


# --------------------------------------------
# Spark Session 
# --------------------------------------------

def get_spark(app_name: str = "analytics_gold") -> SparkSession:
    
    os.environ["PYSPARK_PYTHON"] = sys.executable
    os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable
    builder = (
        SparkSession.builder
        .appName(app_name)
        .config("spark.sql.session.timeZone", "UTC")
        .config("spark.sql.sources.partitionOverwriteMode", "dynamic")
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
        .config("spark.sql.shuffle.partitions", "8")  # bom para local
    )
    return configure_spark_with_delta_pip(builder).getOrCreate()





In [5]:
# Execução: 
spark = get_spark("analytics_gold")


In [10]:
# Carregamento da Camada Gold :

df_gold = spark.read.format("delta").load(GOLD_PATH.as_posix())
df_gold.printSchema()
df_gold.show(5, truncate=False)

root
 |-- data: date (nullable = true)
 |-- ano: integer (nullable = true)
 |-- mes: integer (nullable = true)
 |-- preco_boi_gordo: double (nullable = true)
 |-- ipca_mensal: double (nullable = true)

+----------+----+---+---------------+-----------+
|data      |ano |mes|preco_boi_gordo|ipca_mensal|
+----------+----+---+---------------+-----------+
|2025-01-01|2025|1  |324.95         |0.16       |
|2025-02-01|2025|2  |319.21         |1.31       |
|2025-03-01|2025|3  |312.47         |0.56       |
|2025-04-01|2025|4  |323.96         |0.43       |
|2025-05-01|2025|5  |308.15         |0.26       |
+----------+----+---+---------------+-----------+
only showing top 5 rows



In [11]:
# =========================
# Insights (lag + variação)
# =========================
from pyspark.sql import functions as F,Window

w = Window.orderBy("data")

insights = (
    df_gold.orderBy("data")
    .withColumn("ipca_ant", F.lag("ipca_mensal").over(w))
    .withColumn("boi_ant",  F.lag("preco_boi_gordo").over(w))
    .withColumn(
        "variacao_ipca",
        (F.col("ipca_mensal") - F.col("ipca_ant")) / F.col("ipca_ant") * 100
    )
    .withColumn(
        "variacao_boi",
        (F.col("preco_boi_gordo") - F.col("boi_ant")) / F.col("boi_ant") * 100
    )
    .drop("ipca_ant", "boi_ant")
)

insights.show(10, truncate=False)


+----------+----+---+---------------+-----------+-------------------+-------------------+
|data      |ano |mes|preco_boi_gordo|ipca_mensal|variacao_ipca      |variacao_boi       |
+----------+----+---+---------------+-----------+-------------------+-------------------+
|2025-01-01|2025|1  |324.95         |0.16       |NULL               |NULL               |
|2025-02-01|2025|2  |319.21         |1.31       |718.7500000000001  |-1.7664256039390707|
|2025-03-01|2025|3  |312.47         |0.56       |-57.25190839694656 |-2.1114626734751267|
|2025-04-01|2025|4  |323.96         |0.43       |-23.214285714285722|3.677153006688626  |
|2025-05-01|2025|5  |308.15         |0.26       |-39.53488372093023 |-4.88023212742314  |
|2025-06-01|2025|6  |313.51         |0.24       |-7.692307692307699 |1.7394126237222178 |
|2025-07-01|2025|7  |299.97         |0.26       |8.333333333333341  |-4.318841504258226 |
|2025-08-01|2025|8  |307.25         |-0.11      |-142.3076923076923 |2.4269093576024177 |
|2025-09-0

## Insights Econômicos: IPCA x Boi Gordo (2025)

A análise utiliza valores defasados (lag) para calcular a variação percentual mensal do IPCA e do preço do boi gordo, permitindo observar tendência, volatilidade e possíveis assimetrias de comportamento entre inflação e commodity agropecuária.

O IPCA apresentou elevada volatilidade percentual, especialmente em meses com valores absolutos baixos no período anterior.

O preço do boi gordo mostrou variações mais suaves, com oscilações negativas e positivas mais consistentes ao longo do tempo.

Não há uma correlação direta imediata entre variações mensais do IPCA e do boi gordo, indicando que o preço da commodity responde a outros vetores além da inflação corrente.

## Destaques por Período:

> Janeiro → Fevereiro (2025-01 → 2025-02):

**IPCA**: salto de 0,16% para 1,31%, resultando em uma variação de +718,75%.
**Boi gordo**: queda de -1,77%.

**Insight**: choque inflacionário não foi repassado ao preço do boi no curto prazo.

> Fevereiro → Março:

**IPCA**: desaceleração forte (-57,25%).
**Boi gordo**: nova retração (-2,11%).

**Insight**: enquanto a inflação arrefece, a commodity mantém trajetória de queda com possível influência de fatores sazonais ou oferta elevada.

> Março → Abril:

**IPCA**: nova desaceleração (-23,21%).
**Boi gordo**: alta relevante (+3,68%).

**Insight**: movimento de recuperação do boi ocorre mesmo com inflação em queda, reforçando a desconexão estrutural de curto prazo.

> Abril → Maio:

**IPCA**: continuidade da desaceleração (-39,53%).
**Boi gordo**: forte queda (-4,88%).

**Insight**: período de ajuste simultâneo, mas sem evidência de causalidade direta.

> Maio → Junho

**IPCA**: estabilidade relativa (-7,69%).
**Boi gordo**: leve recuperação (+1,74%).

**Insight**: sinal de piso no preço da commodity.

> Julho → Agosto:

**IPCA**: deflação (-0,11%), gerando variação percentual extrema (-142,31%).

**Boi gordo**: alta de +2,43%.

**Insight**: meses com IPCA negativo ampliam artificialmente a variação percentual, exigindo cautela na interpretação.

Observações Técnicas Importantes

Variações percentuais do IPCA são altamente sensíveis quando o valor do mês anterior é próximo de zero.

### Conclusão Analítica

*O preço do boi gordo não responde de forma imediata às variações mensais do IPCA, indicando que fatores como oferta, demanda externa, ciclo pecuário e sazonalidade têm peso maior no curto prazo do que a inflação oficial.*

In [12]:
# =========================
# Gold Analítico
# =========================
from pyspark.sql import functions as F,Window

analitico = (
    insights
    .withColumn(
        "media_variacoes",
        F.round((F.col("variacao_ipca") + F.col("variacao_boi")) / 2, 2)
    )
    .withColumn(
        "destaque",
        F.when(F.col("variacao_boi") > F.col("variacao_ipca"), "Preço do boi cresce mais")
         .when(F.col("variacao_ipca") > F.col("variacao_boi"), "Inflação cresce mais")
         .otherwise("Empate")
    )
    .withColumn(
        "classe_impacto",
        F.when(F.abs(F.col("variacao_boi") - F.col("variacao_ipca")) > 5, "Alta divergência")
         .when(F.abs(F.col("variacao_boi") - F.col("variacao_ipca")).between(2, 5), "Média divergência")
         .otherwise("Baixa divergência")
    )
)

analitico.show(10, truncate=False)

+----------+----+---+---------------+-----------+-------------------+-------------------+---------------+------------------------+-----------------+
|data      |ano |mes|preco_boi_gordo|ipca_mensal|variacao_ipca      |variacao_boi       |media_variacoes|destaque                |classe_impacto   |
+----------+----+---+---------------+-----------+-------------------+-------------------+---------------+------------------------+-----------------+
|2025-01-01|2025|1  |324.95         |0.16       |NULL               |NULL               |NULL           |Empate                  |Baixa divergência|
|2025-02-01|2025|2  |319.21         |1.31       |718.7500000000001  |-1.7664256039390707|358.49         |Inflação cresce mais    |Alta divergência |
|2025-03-01|2025|3  |312.47         |0.56       |-57.25190839694656 |-2.1114626734751267|-29.68         |Preço do boi cresce mais|Alta divergência |
|2025-04-01|2025|4  |323.96         |0.43       |-23.214285714285722|3.677153006688626  |-9.77          |P

## Insights Analíticos – IPCA x Boi Gordo (Camada Gold)

Esta seção apresenta os insights analíticos derivados da comparação mensal entre o IPCA e o preço do boi gordo, utilizando métricas de variação percentual, média das variações e classificação automática de impacto.

A análise tem como objetivo identificar divergências de comportamento entre inflação oficial e commodity agropecuária ao longo do tempo.

Para cada período mensal, foram calculados a variação percentual do IPCA, variação percentual do preço do boi gordo
e média das variações (media_variacoes) como indicador de assimetria.

Destaque indicando qual variável apresentou maior impacto relativo.

Em praticamente todos os meses analisados, a relação entre IPCA e boi gordo foi classificada como ```Alta divergência.```
> Isso indica que as duas séries não apresentam sincronização mensal, reforçando que o preço do boi gordo possui dinâmica própria.

O preço do boi cresce mais, tendo comportamento dominante na maior parte do período.
A inflação cresce mais em ocorrências pontuais.
Há empate apenas no mês inicial (sem cálculo de variação).

> O boi gordo não reage de forma direta ou imediata às variações mensais do IPCA.

A coluna ```media_variacoes``` funciona como um termômetro de divergência entre inflação e commodity:

 - Valores positivos elevados indicam dominância da inflação
 - Valores negativos acentuados indicam maior impacto relativo do boi gordo
 - Valores próximos de zero indicam equilíbrio ou baixa intensidade

*Eventos extremos (como meses com IPCA muito baixo ou negativo) podem gerar variações percentuais elevadas, exigindo cautela na interpretação econômica, apesar de estarem estatisticamente corretos.*

> Meses classificados como “Inflação cresce mais” representam choques inflacionários que não foram repassados ao preço do boi no curto prazo. E meses classificados como “Preço do boi cresce mais” mostram que a commodity reage a fatores próprios do setor agropecuário que corrresponde a oferta, demanda, exportações e ciclo pecuário e não depende exclusivamente da inflação oficial.

A predominância da classe Alta divergência indica baixa correlação mensal entre IPCA e boi gordo.
Necessidade de análises complementares, como médias móveis, comparações interanuais e modelos multivariados.

> O preço do boi gordo apresenta comportamento predominantemente independente da inflação mensal, com divergência elevada ao longo de quase todo o período analisado.



In [ ]:
from pyspark.sql import functions as F
import math

# (opcional) remover linhas com variação nula, evita corr/média estranhas
base_metrics = analitico.filter(
    F.col("variacao_ipca").isNotNull() & F.col("variacao_boi").isNotNull()
)

# Correlação (retorna float/None)
corr = base_metrics.stat.corr("variacao_ipca", "variacao_boi")
if corr is not None and isinstance(corr, float) and math.isnan(corr):
    corr = None

# Médias (retorna DataFrame)
medias_df = base_metrics.agg(
    F.avg("variacao_ipca").alias("media_ipca"),
    F.avg("variacao_boi").alias("media_boi")
)

# Mostrar as médias
medias_df.show(truncate=False)

# Pegar valores como números Python
medias = medias_df.first()
media_ipca = float(medias["media_ipca"]) if medias["media_ipca"] is not None else None
media_boi  = float(medias["media_boi"])  if medias["media_boi"]  is not None else None
corr_val   = float(corr) if corr is not None else None

# Adicionar métricas globais em todas as linhas
analitico = (
    analitico
    .withColumn("correlacao_global", F.lit(corr_val))
    .withColumn("media_ipca_global", F.lit(media_ipca))
    .withColumn("media_boi_global",  F.lit(media_boi))
    .withColumn(
        "media_combinada_global",
        F.round((F.col("media_ipca_global") + F.col("media_boi_global")) / 2, 4)
    )
)

# Mostrar resultado final (incluindo métricas globais)
analitico.select(
    "data", "variacao_ipca", "variacao_boi",
    "correlacao_global", "media_ipca_global", "media_boi_global", "media_combinada_global"
).show(10, truncate=False)


+------------------+--------------------+
|media_ipca        |media_boi           |
+------------------+--------------------+
|2.0729047700789014|-0.09153922145628818|
+------------------+--------------------+

+----------+-------------------+-------------------+-------------------+------------------+--------------------+----------------------+
|data      |variacao_ipca      |variacao_boi       |correlacao_global  |media_ipca_global |media_boi_global    |media_combinada_global|
+----------+-------------------+-------------------+-------------------+------------------+--------------------+----------------------+
|2025-01-01|NULL               |NULL               |-0.1488842064048268|2.0729047700789014|-0.09153922145628818|0.9907                |
|2025-02-01|718.7500000000001  |-1.7664256039390707|-0.1488842064048268|2.0729047700789014|-0.09153922145628818|0.9907                |
|2025-03-01|-57.25190839694656 |-2.1114626734751267|-0.1488842064048268|2.0729047700789014|-0.091539221456288

## Insights Globais – Métricas Estatísticas (Camada Analítica)

Além dos insights mensais (lag/variações e classificação de impacto), o projeto calcula métricas globais a partir das séries de variação percentual do IPCA e do preço do boi gordo.
Para evitar distorções, as métricas são calculadas apenas em linhas onde variacao_ipca e variacao_boi não são nulas.

### Médias Globais das Variações:

- Média da variação do IPCA (media_ipca_global): **2,0729%**
- Média da variação do boi gordo (media_boi_global): **-0,0915%**
- Média combinada (media_combinada_global): **0,9907%**

> No agregado do período analisado, o IPCA apresentou variação média positiva, indicando tendência inflacionária média no recorte. Já o boi gordo apresentou variação média ligeiramente negativa, sugerindo que, em média, o preço ficou estável com viés de queda leve no período. A média combinada serve como um indicador sintético do “movimento médio conjunto”, mas deve ser interpretada com cautela pois mistura grandezas com dinâmicas diferentes.

### Correlação Global entre as Séries:

Correlação global (correlacao_global) entre ```variacao_ipca``` e ```variacao_boi```: **-0,1489**

> A correlação é fraca e negativa, o que indica que não existe relação linear forte entre as variações mensais do IPCA e do boi gordo. Quando uma sobe, a outra tende levemente a cair, mas o efeito é pequeno (quase neutro).

*Isso reforça que o boi gordo responde majoritariamente a fatores próprios do setor (oferta, sazonalidade, ciclo pecuário, exportações), e não diretamente ao comportamento mensal do IPCA.*

In [ ]:
# =========================
# Escrita Delta
# =========================
from pyspark.sql import functions as F,Window

(insights.write.format("delta").mode("overwrite").save(INSIGHTS_PATH.as_posix()))
(analitico.write.format("delta").mode("overwrite").save(ANALITICO_PATH.as_posix()))


In [ ]:
# Criação do database gold :

spark.sql(f"""
CREATE DATABASE IF NOT EXISTS gold
LOCATION '{GOLD_ROOT.as_posix()}'
""")


DataFrame[]

In [ ]:
spark.sql("SHOW DATABASES").show(truncate=False)


+---------+
|namespace|
+---------+
|default  |
|gold     |
+---------+



In [ ]:
# =========================
# Registro no catálogo
# =========================
spark.sql(f"CREATE TABLE IF NOT EXISTS gold.insights USING DELTA LOCATION '{INSIGHTS_PATH.as_posix()}'")
spark.sql(f"CREATE TABLE IF NOT EXISTS gold.analitico USING DELTA LOCATION '{ANALITICO_PATH.as_posix()}'")

DataFrame[]

In [ ]:
# Validações:

spark.sql("SHOW TABLES IN gold").show(truncate=False)
spark.table("gold.insights").show(5, truncate=False)
spark.table("gold.analitico").show(5, truncate=False)

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|gold     |analitico|false      |
|gold     |insights |false      |
+---------+---------+-----------+

+----------+----+---+---------------+-----------+-------------------+-------------------+
|data      |ano |mes|preco_boi_gordo|ipca_mensal|variacao_ipca      |variacao_boi       |
+----------+----+---+---------------+-----------+-------------------+-------------------+
|2025-01-01|2025|1  |324.95         |0.16       |NULL               |NULL               |
|2025-02-01|2025|2  |319.21         |1.31       |718.7500000000001  |-1.7664256039390707|
|2025-03-01|2025|3  |312.47         |0.56       |-57.25190839694656 |-2.1114626734751267|
|2025-04-01|2025|4  |323.96         |0.43       |-23.214285714285722|3.677153006688626  |
|2025-05-01|2025|5  |308.15         |0.26       |-39.53488372093023 |-4.88023212742314  |
+----------+----+---+---------------+-----------+-------------------+------

In [ ]:
# =========================
# View GOLD (dashboard)
# =========================
spark.sql("""
CREATE OR REPLACE VIEW gold.vw_gold_dashboard AS
SELECT * FROM gold.analitico
""")

# =========================
# teste:
# =========================
spark.table("gold.vw_gold_dashboard") \
     .select("data", "variacao_ipca", "variacao_boi", "destaque", "classe_impacto") \
     .show(20, truncate=False)

+----------+-------------------+-------------------+------------------------+-----------------+
|data      |variacao_ipca      |variacao_boi       |destaque                |classe_impacto   |
+----------+-------------------+-------------------+------------------------+-----------------+
|2025-01-01|NULL               |NULL               |Empate                  |Baixa divergência|
|2025-02-01|718.7500000000001  |-1.7664256039390707|Inflação cresce mais    |Alta divergência |
|2025-03-01|-57.25190839694656 |-2.1114626734751267|Preço do boi cresce mais|Alta divergência |
|2025-04-01|-23.214285714285722|3.677153006688626  |Preço do boi cresce mais|Alta divergência |
|2025-05-01|-39.53488372093023 |-4.88023212742314  |Preço do boi cresce mais|Alta divergência |
|2025-06-01|-7.692307692307699 |1.7394126237222178 |Preço do boi cresce mais|Alta divergência |
|2025-07-01|8.333333333333341  |-4.318841504258226 |Inflação cresce mais    |Alta divergência |
|2025-08-01|-142.3076923076923 |2.426909

In [ ]:
spark.stop()